In [2]:

%matplotlib inline
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics
import pdf2docx

import pickle

import camelot
import typing
import io
import numpy as np
from docx import Document
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
%matplotlib widget
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format=u'%(filename)+13s [ LINE:%(lineno)-4s] %(levelname)-8s %(message)s',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)

from docx2python import docx2python
import traceback
import io


In [5]:

class HeaderFinder:

    def find_and_split_departments(self, df: pd.DataFrame) -> pd.DataFrame: # отдает df с департаментом, если он был в таблице 

                    
        def _add_department_info_to_df(df: pd.DataFrame, dep_info: list[dict[int, str]]) -> pd.DataFrame:
            df['department'] = None
            for data in dep_info: 
                df.at[data['index'], 'department'] = data['dep']       
            df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=0)
            return df


        # table splitter
        def _find_department_in_table(df: pd.DataFrame) -> list[dict[int, str]]: # [index, department]
            departments_n_indexes = []
            for row in df.itertuples():
                index = row[0]
                row = list(row)[1:-1]
                row = [e for e in row if len(str(e)) > 4]
                if len(set(row))<2:
                    if not all([type(e) in [int, float] for e in row]):  
                        if statistics.mean([len(e) for e in row]) > 4:
                            departments_n_indexes.append({'index':index, 'dep': row[0]})
            return departments_n_indexes

        dep_info = _find_department_in_table(df)
        
        if not dep_info:
            return df
        
        # добавили колонку с департаментом    
        df = _add_department_info_to_df(df, dep_info)
        # удалили одинаковые ряды
        # df = ''
        return df



header_finder = HeaderFinder()
# data_cleaner = DataCleaner()

# r = header_finder.find_and_split_departments(df)
# data_cleaner.drop_same_cols(r)

# indx = _find_department_in_table(df)
# indx
# df


In [81]:



class DocxParser:

    def get_docx_tables(self, filename, tab_id=None, **kwargs) -> list[pd.DataFrame]:
        """
            filename:   file name of a Word Document
            tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                        When [None] - return a list of DataFrames (parse all tables)
        """
        def read_docx_tab(tab, **kwargs):
            vf = io.StringIO()
            writer = csv.writer(vf)
            for row in tab.rows:
                writer.writerow(cell.text for cell in row.cells)
            vf.seek(0)
            return pd.read_csv(vf, header=None, **kwargs)

        doc = Document(filename)
        if tab_id is None:
            return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
        else:
            try:
                return read_docx_tab(doc.tables[tab_id], **kwargs)
            except IndexError:
                print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
                raise
            

    def convert_docx_to_df(self, filename: str) -> pd.DataFrame:
        assert filename.endswith('docx'), 'Формат должен быть .docx!'
            
        doc = Document(filename)
        # TODO: тут взять текст, который потом прикрутить к

        doc_tables = self.get_docx_tables(filename) 
        
        return doc_tables


    @staticmethod
    def find_ok_cols(cols: tuple) -> Union[dict[int, str], None]: # отдает словарь с колонками, которые надо переименовать
        '''ищем заголовки в таблице'''
        
        cols = list(map(str, cols))
        cols = list(map(str.lower, cols))
        if len(set(cols)) == 1:
            return 

        ok_cols = 0
        result = {} # 'n_of_col':'new_name'
        for i, col in enumerate(cols):
            if len(result) > 5 or i > 20:
                return result

            if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col) and 'name' not in result.values():
                
                result[i] = 'name'
                ok_cols+=1

            elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, е, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col): #and "salary" not in result.values():
                
                result[i] = "salary"
                ok_cols+=1
                
            elif re.search(pattern='(должност[ь, и, ей])', string=col) and 'position' not in result.values(): 
                
                result[i] = 'position'
                ok_cols+=1
                
            elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическ|организаци|наименование [оу, мо])', string=col) and 'department' not in result.values():
                print('col---',col)
                
                result[i] = 'department'
                ok_cols+=1

        if len(result) >= 2:
            return result
        return False


    def detect_headers(self, df: pd.DataFrame)-> dict[bool, pd.DataFrame]: # словарь с результатом найденных заголовков
        # сместили колонки в ряды        
        # df.loc[-1] = df.columns  # adding a row
        # df.index = df.index + 1  # shifting index
        # df = df.sort_index()  # sorting by index

        for row in df.itertuples():
            index = row[0] 
            new_cols = self.find_ok_cols(row[1:])
            if new_cols:
                break

        if not new_cols:
            return {'ok_cols':False, 'df':df}

        for k,v in new_cols.items():
            df.rename(columns={df.columns[k]:v}, inplace=True)

        df = df.iloc[index:,:]
        return {'ok_cols':True, 'df':df}


    def concatenate_if_possible(self, dfs: list[dict['df':pd.DataFrame, 'ok_cols':bool]]) -> list[pd.DataFrame]:

        all_oks = [e['ok_cols'] for e in dfs]

        if all(all_oks):
            return [e['df'] for e in dfs]

        result_df = []
        df_to_concat = pd.DataFrame()
        for df_info in dfs:
            if df_info['ok_cols']:
                if not df_to_concat.empty:
                    result_df.append(df_to_concat)
                df_to_concat = df_info['df']

            # оставляем только таблицы, у которых совпадает число колонок
            # с df у которых мы колонки нашли
            # если не нашли колонки и не к чему присоединять - дропаем

            elif not df_info['ok_cols'] and not df_to_concat.empty \
                    and len(df_to_concat.columns) == len(df_info['df'].columns):
                df_info['df'].columns = df_to_concat.columns
                df_to_concat = pd.concat([df_to_concat, df_info['df']])

        result_df.append(df_to_concat)

        return result_df

    def rename_df_col(self, df: pd.DataFrame) -> pd.DataFrame:
        
        # for row in df.itertuples():
        #     index = row[0] 
        #     new_cols = self.find_ok_cols(row[1:])
        #     if new_cols:
        #         break
        # df.columns
        
        cols = df.columns
        cols = list(map(str.lower, cols))

        for i,col in enumerate(cols):
            if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col):
                print('col---',col)
                cols[i] = "name"

            elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col):
                print('col---',col)
                cols[i] = "salary"

            elif re.search(pattern='(должност[ь, и, ей])', string=col):
                print('col---',col)
                cols[i] = 'position'

            elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическ|организаци)', string=col):
                print('col---',col)
                cols[i] = 'department'
        
        df.columns = cols
        return df
    
    @staticmethod
    def get_offices_from_doc(filename:str) -> list[str]:

        doc = docx2python(filename)

        table_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|должность)'

        offices = []
        gathering_office_info = ''

        for paragraph in doc.body_runs:  # параграфы в виде вложенных листов

            paragraph = sum(sum(paragraph, []), [])
            paragraph_text = ''
            for e in paragraph:
                try:
                    paragraph_text += ' ' + e[0] + ' '
                except IndexError:
                    pass

            paragraph_text = paragraph_text.lower()
            its_table = re.findall(
                pattern=table_pattern, string=paragraph_text)

            if not its_table:
                gathering_office_info += paragraph_text

            elif its_table:
                offices.append(gathering_office_info)
                gathering_office_info = ''

        return [e for e in offices if e]

    @staticmethod
    def compile_office_info_and_df(departments: list, tables: list[pd.DataFrame]) -> typing.Union[None, list[pd.DataFrame]]:

        ok_dfs = []

        print('Количество заголовков --- ', len(departments))
        print('Количество таблиц --- ', len(tables))

        if len(departments) - len(tables) == 1:
            departments.pop()

        if len(departments) == len(tables):
            for table, dep in zip(tables, departments):
                table['department'] = dep
                table['department'][0] = 'Учреждение'
                ok_dfs.append(table)

            return ok_dfs

        raise ValueError(
            f'Разное число таблиц ({len(tables)}) и учреждений ({len(departments)})')


    def parse_file(self, filepath:str):
        dfs = self.get_docx_tables(filepath)
        dfs = [self.detect_headers(df) for df in dfs]
        dfs = self.concatenate_if_possible(dfs)
        # dfs = [self.rename_df_col(df) for df in dfs]
        
        how_many_dfs_with_departments = sum([1 for e in dfs if 'department' in e.columns])
        # если учреждений больше, чем у половины таблиц, то отдаем так
        if how_many_dfs_with_departments > len(dfs) // 2 :
            return dfs

        else:

            dfs = [header_finder.find_and_split_departments(df) for df in dfs]

            how_many_dfs_with_departments = sum([1 for e in dfs if 'department' in e.columns])
            # если учреждений больше, чем у половины таблиц, то отдаем так
            if how_many_dfs_with_departments > len(dfs) // 2 :
                return dfs

            departments = self.get_offices_from_doc(filepath)
            dfs = self.compile_office_info_and_df(tables=dfs, departments=departments)
            return dfs




# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\83299_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\83310_2016_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
# #file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\83292_2016_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\docx\83316_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\83310_2016_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\83349_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii_(ne_ukazany_FIO).docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\83364_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\188073_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(obrazovanie).docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\aaa.docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\187331_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii.docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\84903_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\84548_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"

# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\83310_2016_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.docx"

parser = DocxParser()

# res = parser.parse_file(file)
# res[0]

        

In [83]:
folder = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx'
# result_folder = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\parsed'
result_folder = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\results\ok_docx'
errors = []

for file in tqdm_notebook(os.listdir(folder)):
    if file.endswith('.docx'):
        try:
            filepath = folder + '//' + file
            res = parser.parse_file(filepath)
            for i, table in enumerate(res):
                table.to_excel(result_folder + '//' + f'{i}_' + file.strip('.docx')+'.xlsx')    
        except Exception as ex:            
            buffer = io.StringIO()
            traceback.print_exc(file=buffer)
            buffer.seek(0)
            errors.append({'file':file, 'err':buffer.readlines()})



In [55]:
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\187692_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii.docx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx\187741_2020_Rukovoditeli_podvedomstvennykh_uchrezhdenii_(kul'tura).docx"
# res = parser.par(file)
print(res[0]['Учреждение'].values)


[' приложение  информация о среднемесячной заработной плате за 2020 год   информация о среднемесячной заработной плате за 2020 год   муп «транс-сервис»  '
 ' приложение  информация о среднемесячной заработной плате за 2020 год   информация о среднемесячной заработной плате за 2020 год   муп «транс-сервис»  ']


In [38]:
l = [1,2,3]
for e in l:
    e = 1
l

[1, 2, 3]

In [95]:
errors

37

In [11]:
'''
1. Все таблицы
2. У всех ли таблиц есть заголовки
3. Если у некоторых есть, а у других нет и одинаковое кол-во колонок - можно соединить
4. Если нет учреждения:
    а. пытаемся сделать сплит
    б. добавляем весь текст перед таблицей (в классе должен храниться распаршенный вариант)

'''
# я хочу посмотреть у скольких доков не ок заги


#folder = r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\all_docx'
# df = res[0].copy()
move_cols_to_rows(df)


,№ п/п,"Фамилия, имя, отчество",Должность,"Среднемесячная заработная плата,\nрублей"
0,№ п/п,"Фамилия, имя, отчество",Должность,"Среднемесячная заработная плата,\nрублей"
1,2,3,4,aaaaaa
2,1,2,3,4
3,1. Департамент по недропользованию по Централь...,1. Департамент по недропользованию по Централь...,1. Департамент по недропользованию по Централь...,1. Департамент по недропользованию по Централь...
4,1.,Савицкий М.Ф.,Начальник,"98 249,50"
...,...,...,...,...
65,10. Управление по недропользованию по Республи...,10. Управление по недропользованию по Республи...,10. Управление по недропользованию по Республи...,10. Управление по недропользованию по Республи...
66,1.,Лацановский И.А.,Начальник,"131 261,04"
67,2.,Гаевая А.В.,Заместитель начальника,"109 796,82"
68,3.,Оленица Т.В.,Заместитель начальника,"62 950,48"
